In [ ]:
from google.colab import files
import zipfile,os,glob,re
import pandas as pd
from datetime import datetime

#機能0：Colab Notebooksにsamplesに入っているファイルをZIP形式で読み込む

#samplesのZIPファイルを選択してアップロード
uploaded = files.upload()

##アップロードしたZipファイルを「unzipped_samples」フォルダの配下に解凍
with zipfile.ZipFile("samples.zip", "r") as zip_ref:
    zip_ref.extractall("unzipped_samples")



Saving samples.zip to samples.zip


In [ ]:
# F001: 注文集計機能

class OrderAggregator:
    """注文ファイルを読み込み、日別の注文合計を集計するクラス"""
    def __init__(self, folder_path, file_pattern="order_*.xlsx", sheet_name="Sheet1"):
        self.folder_path  = folder_path     # 注文ファイルのフォルダパス
        self.file_pattern = file_pattern    # ファイル名パターン（例：order_*.xlsx）
        self.sheet_name   = sheet_name      # 読み込むシート名

    def summarize_by_date(self):            # 注文ファイルを読み込み、注文日ごとに商品別注文数を集計して返す
        files   = glob.glob(os.path.join(self.folder_path, self.file_pattern))  # 注文ファイル一覧を取得
        df_list = []
        for f in files:
            match = re.search(r"(\d{8})", f)                                    # ファイル名から注文日（8桁）を抽出
            df    = pd.read_excel(f, sheet_name=self.sheet_name).fillna(0)      # 注文データを読み込み
            df["注文日"] = pd.to_datetime(match.group(1), format="%Y%m%d")      # 注文日を追加
            df_list.append(df)
        return pd.concat(df_list, ignore_index=True).groupby("注文日").sum(numeric_only=True)  # 注文日ごとに集計

aggregator     = OrderAggregator("unzipped_samples/samples/order_new")          # 注文ファイルのフォルダパスを渡して初期化
daily_summary  = aggregator.summarize_by_date()                                 # 注文日ごとの集計結果を取得
print(daily_summary)


# F002: 在庫確認機能

class InventoryManager:
    """在庫表の最終行を抽出し、最新の野菜の在庫情報を取得するクラス"""
    def __init__(self, folder_path, file_name="inventory.xlsx"):
        self.file_path = os.path.join(folder_path, file_name)                   # 在庫ファイルのパス

    def get_latest_inventory(self):                                             # 在庫表の最終行（最新の在庫情報）を取得して返す
        df = pd.read_excel(self.file_path)
        return df.iloc[-1]                                                      # 最終行を返す

manager          = InventoryManager("unzipped_samples/samples")                 # 在庫ファイルのフォルダパスを渡して初期化
latest_inventory = manager.get_latest_inventory()                               # 最新の在庫情報を取得
print(latest_inventory)


# F003: しきい値確認機能

class ThresholdLoader:
    """ 発注基準となるしきい値と追加量をpickup.xlsxから取得するクラス"""
    def __init__(self, folder_path, file_name="pickup.xlsx"):
        self.file_path = os.path.join(folder_path, file_name)                   # しきい値ファイルのパス

    def load_thresholds(self):                                                  # Excelファイルから「しきい値」「追加量」を辞書形式で取得して返す
        df         = pd.read_excel(self.file_path, index_col=0)
        thresholds = df.loc["しきい値"].to_dict()                               # 各野菜のしきい値を取得
        additions  = df.loc["追加量"].to_dict()                                 # 各野菜の追加量を取得
        return {"しきい値": thresholds, "追加量": additions}

loader = ThresholdLoader("unzipped_samples/samples")                            # しきい値ファイルのフォルダパスを渡して初期化
data   = loader.load_thresholds()                                               # しきい値と追加量を取得
print("しきい値:\n", data["しきい値"])                                          # しきい値を表示
print("追加量:\n", data["追加量"])

# F004: 発注判定機能

class PurchasePlanner:
    """在庫・注文・しきい値をもとに残在庫数を計算し、発注が必要な野菜を抽出するクラス"""
    def __init__(self, inventory_folder, order_folder, threshold_folder):
        self.inventory  = InventoryManager(inventory_folder)                    # 在庫管理クラスを初期化
        self.orders     = OrderAggregator(order_folder)                         # 注文集計クラスを初期化
        self.thresholds = ThresholdLoader(threshold_folder).load_thresholds()   # しきい値データを取得

    def latest_inventory(self):                                                 # 最新の在庫情報を取得
        return self.inventory.get_latest_inventory()

    def total_orders(self):                                                     # 各野菜の合計注文数を取得
        return self.orders.summarize_by_date().sum(numeric_only=True)

    def plan(self):                                                             # 在庫・注文・しきい値・追加量を統合し、残在庫数を計算した一覧を返す
        inv   = self.latest_inventory().drop(labels="日付", errors="ignore")    # 「日付」列を除外（野菜名のみ残す）
        orders= self.total_orders()                                             # 合計注文数を取得
        plan = pd.DataFrame({
          "在庫数": inv,
          "注文合計": orders,
          "残在庫数": inv - orders,
          "しきい値": pd.Series(self.thresholds["しきい値"]).reindex(inv.index),
          "追加量": pd.Series(self.thresholds["追加量"]).reindex(inv.index),
        })
        plan = plan.drop(index="曜日", errors="ignore")                         # 「曜日」行が存在する場合は除外（不要な行の削除）
        return plan                                                             # 発注判定に必要な一覧を返す

    def pickup_items(self):                                                     # 残在庫数がしきい値を下回る野菜のみ抽出し、追加量を返す
        plan   = self.plan()
        pickup = plan[plan["残在庫数"] < plan["しきい値"]]                      # 発注対象の野菜を抽出
        return pickup[["追加量"]]                                               # 発注対象の追加量のみ返す

planner = PurchasePlanner("unzipped_samples/samples",
                          "unzipped_samples/samples/order_new",
                          "unzipped_samples/samples")                           # 各ファイルのフォルダパスを渡して初期化

print("全野菜の在庫状況:\n", planner.plan())                                    # 在庫・注文・しきい値・追加量の一覧を表示
print("\n発注対象:\n", planner.pickup_items())

# F005: メール作成機能

class MailComposer:
    """発注対象の野菜と追加量をもとに、メール本文をテンプレート形式で生成するクラス"""
    def __init__(self, planner, farm_name="〇〇農園", wholesaler_name="〇〇卸売株式会社"):
        self.planner = planner                                                  # 発注判定機能を保持
        self.farm_name = farm_name                                              # 宛先農家名
        self.wholesaler_name = wholesaler_name                                  # 発注者

    def compose_email(self):                                                    # 発注内容に応じたメール本文を生成して返す
        pickup_items = self.planner.pickup_items()                              # 発注対象の野菜と追加量を取得

        # 発注がない場合のテンプレート
        if pickup_items.empty:
            body = (
                f"{self.farm_name} 御中\n\n"
                "いつもお世話になっております。\n"
                "本日は発注はございません。\n\n"
                f"{self.wholesaler_name}"
            )
            return body

        # 発注がある場合のテンプレート
        body = (
            f"{self.farm_name} 御中\n\n"
            "いつもお世話になっております。\n"
            "以下の野菜について発注します。\n\n"
        )

        for item, row in pickup_items.iterrows():                               # 発注対象の野菜を1行ずつ追加
            body += f"・{item}: {int(row['追加量'])} 個\n"

        body += (
            "\nご対応のほどよろしくお願いいたします。\n\n"
            f"{self.wholesaler_name}"
        )

        return body

planner = PurchasePlanner(
    "unzipped_samples/samples",
    "unzipped_samples/samples/order_new",
    "unzipped_samples/samples"
)

composer = MailComposer(planner, farm_name="さむらい農園", wholesaler_name="はなこ青果卸売株式会社")
mail_body = composer.compose_email()

print(mail_body)

# F006: メール送信機能

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

class MailSender:
    """SMTPを使ってメールを送信するクラス"""
    def __init__(self, smtp_server, smtp_port, username, password, from_addr):
        self.smtp_server = smtp_server                                          # SMTPサーバーのホスト名
        self.smtp_port   = smtp_port                                            # SMTPポート番号
        self.username    = username                                             # 認証用ユーザー名
        self.password    = password                                             # 認証用パスワード
        self.from_addr   = from_addr                                            # 送信元メールアドレス

    def send_email(self, to_addr, subject, body):                               # メールを送信する関数
        msg = MIMEMultipart()                                                   # メールオブジェクトを作成
        msg["From"] = self.from_addr
        msg["To"] = to_addr
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain", "utf-8"))                            # 本文を添付

        with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
            server.starttls()                                                   # TLS暗号化を開始
            server.login(self.username, self.password)                          # ログイン
            server.send_message(msg)                                            # メール送信

        print("メールを送信しました:", to_addr)                                 # 送信完了メッセージ

# MailtrapのSMTP情報
SMTP_SERVER = "smtp.mailtrap.io"
SMTP_PORT = 25
USERNAME = "05401b29566577"
PASSWORD = "9c8c8cd6c87344"
FROM_ADDR = "wholesaler@example.com"                                            # 卸売業者のメールアドレス
TO_ADDR = "farmer@example.com"                                                  # 農家のメールアドレス

from datetime import datetime

# 今日の日付を件名にセット
today_str = datetime.today().strftime("%Y-%m-%d")
subject = f"発注依頼（{today_str}）"

# PurchasePlannerとMailComposerを利用して本文生成
planner = PurchasePlanner(
    "unzipped_samples/samples",
    "unzipped_samples/samples/order_new",
    "unzipped_samples/samples"
)
composer = MailComposer(planner, farm_name="さむらい農園", wholesaler_name="はなこ青果卸売株式会社")
mail_body = composer.compose_email()

# 件名を「最新注文日」に修正
daily_summary     = planner.orders.summarize_by_date()
latest_order_date = daily_summary.index.max()
order_date_str    = latest_order_date.strftime("%Y-%m-%d")
subject           = f"発注依頼（{order_date_str}）"

# メール送信
sender = MailSender(SMTP_SERVER, SMTP_PORT, USERNAME, PASSWORD, FROM_ADDR)
sender.send_email(TO_ADDR, subject, mail_body)

# F007: 在庫更新機能

class InventoryUpdater:
    """残在庫数＋追加量を反映してinventory.xlsxに追記するクラス"""
    def __init__(self, folder_path, file_name="inventory.xlsx"):
        self.file_path = os.path.join(folder_path, file_name)                   # 在庫ファイルのパスを設定

    def update_inventory(self, planner):                                        # 在庫を更新してファイルに追記
        df     = pd.read_excel(self.file_path)                                  # 既存の在庫データを読み込み
        plan   = planner.plan()                                                 # 発注計画を取得
        pickup = planner.pickup_items()                                         # 発注対象の野菜を取得

        updated = plan["残在庫数"].copy()                                       # 残在庫数をコピー
        updated.loc[pickup.index] += plan.loc[pickup.index, "追加量"]           # 発注分を加算

        # 注文ファイルから最新注文日を取得（時刻付き）
        daily_summary     = planner.orders.summarize_by_date()
        latest_order_date = daily_summary.index.max()

        new_row = updated.to_dict()                                             # 更新後の在庫を辞書化
        new_row["日付"] = latest_order_date.strftime("%Y-%m-%d %H:%M:%S")       # 日付を追加
        new_row["曜日"] = latest_order_date.strftime("%a")                      # 曜日を追加

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)        # 新しい行を追加
        df.to_excel(self.file_path, index=False)                                # Excelファイルに保存

        return new_row

# PurchasePlannerを利用して最新の在庫計画を取得
planner = PurchasePlanner(
    "unzipped_samples/samples",
    "unzipped_samples/samples/order_new",
    "unzipped_samples/samples"
)

# 在庫更新クラスを呼び出し
updater = InventoryUpdater("unzipped_samples/samples")

# 在庫を更新（残在庫数＋追加量）
updated_inventory = updater.update_inventory(planner)

print("更新後の在庫状況:\n", updated_inventory)

             トマト  キャベツ  レタス  ほうれん草  ニンジン    白菜    大根
注文日                                                 
2023-05-24  31.0  21.0   42   23.0  32.0  25.0  15.0
日付       2023-05-14 00:00:00
曜日                       Sun
トマト                       91
キャベツ                      73
レタス                      103
白菜                        84
ほうれん草                     75
大根                        48
ニンジン                      50
Name: 13, dtype: object
しきい値:
 {'トマト': 50, 'キャベツ': 40, 'レタス': 50, '白菜': 30, 'ほうれん草': 40, '大根': 30, 'ニンジン': 40}
追加量:
 {'トマト': 100, 'キャベツ': 80, 'レタス': 100, '白菜': 60, 'ほうれん草': 80, '大根': 60, 'ニンジン': 80}
全野菜の在庫状況:
        在庫数  注文合計  残在庫数  しきい値    追加量
ほうれん草   75  23.0  52.0  40.0   80.0
キャベツ    73  21.0  52.0  40.0   80.0
トマト     91  31.0  60.0  50.0  100.0
ニンジン    50  32.0  18.0  40.0   80.0
レタス    103  42.0  61.0  50.0  100.0
大根      48  15.0  33.0  30.0   60.0
白菜      84  25.0  59.0  30.0   60.0

発注対象:
        追加量
ニンジン  80.0
さむらい農園 御中

いつもお世話になっております。
以下の野菜について発注します。

・ニンジン: 8